# Functions

In [1]:
from datetime import time
import datetime as dt

def adddatecolumns(data,pf,column) :
    data['year'] = pf[column].apply(lambda x: x.date().year)
    data['month'] = pf[column].apply(lambda x: x.date().month)
    data['day'] = pf[column].apply(lambda x: x.date().day)
    data['hour'] = pf[column].apply(lambda x: x.time().hour)
    data['minute'] = pf[column].apply(lambda x: x.time().minute)
    #data['second'] = pf[column].apply(lambda x: x.time().second)
    #data['microsecond'] = pf[column].apply(lambda x: x.time().microsecond)

def converttimestampcolumnn(pf,tsc) :
    pf[tsc] = pf[tsc].apply(lambda x: dt.datetime.fromtimestamp(float(x) / 1e3))

In [2]:
## Setup charts
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

#vis functions

def label(graph,skip,rot) :
    for ind, label in enumerate(graph.get_xticklabels()):
        if ind % skip == 0:  # every 10th label is kept
            label.set_visible(True)
            label.set_rotation(rot)
        else:
            label.set_visible(False)
            
def abc(mdcountsall) :
    a = mdcountsall.index.get_level_values(0).astype(str)
    b = mdcountsall.index.get_level_values(1).astype(str)
    c = mdcountsall.index.get_level_values(2).astype(str)
    return a + "-" + b + "-" + c 

def abcd(mdcountsall) :
    return abc(mdcountsall) + "-" + mdcountsall.index.get_level_values(3).astype(str)

def ymdh(mdcountsall) :
    return abcd(mdcountsall)

def get_ymdh(mdcountsall) :
    a = mdcountsall.index.get_level_values(0).astype(str)
    b = mdcountsall.index.get_level_values(1).astype(str)
    c = mdcountsall.index.get_level_values(2).astype(str)
    d = mdcountsall.index.get_level_values(3).astype(str)
    return a,b,c,d

def get_ymdh_string(a,b,c,d) :
    return a + "-" + b + "-" + c + "-" + d

def get_ym(mdcountsall) :
    a = mdcountsall.index.get_level_values(0).astype(str)
    b = mdcountsall.index.get_level_values(1).astype(str)
    return a,b

def get_ym_string(a,b) :
    return a + "-" + b

Setup Complete


In [3]:

#mdcountsall=pfall[(pfall['month'] == 2) & (pfall['day'] > 16)].groupby(['year','month','day','hour'])['outcome'].count()
#mdcountsall=pfall[(pfall['month'] == 2)].groupby(['year','month','day','hour'])['outcome'].count()
#mdcountsall=pfall.groupby(['year','month','day'])['outcome'].count()

def createData(pfall,month,outcome) :
    if outcome < 2 :
        mdcountsall = pfall[(pfall['month'] == month) & (pfall['outcome'] == outcome)].groupby(['year','month','day','hour'])['outcome'].count()
    else :
        if (month > 0) & (month < 13) :
            mdcountsall = pfall[(pfall['month'] == month)].groupby(['year','month','day','hour'])['outcome'].count()
        else :
            mdcountsall = pfall.groupby(['year','month','day','hour'])['outcome'].count()    
    a,b,c,d = get_ymdh(mdcountsall)
    data2 = pd.DataFrame()
    data2['date'] = get_ymdh_string(a,b,c,d)
    data2['outcome'] =  mdcountsall.reset_index()['outcome'].astype(int) 

    #for pivot table
    data2['hours'] =  d.astype(int) 
    data2['days']  =  c.astype(int) 
    return data2

def createData_ym(pfall,month,outcome) :
    if outcome < 2 :
        mdcountsall = pfall[(pfall['outcome'] == outcome)].groupby(['year','month'])['outcome'].count()
    else :
        if (month > 0) & (month < 13) :
            mdcountsall = pfall[(pfall['month'] == month)].groupby(['year','month'])['outcome'].count()
        else :
            mdcountsall = pfall.groupby(['year','month'])['outcome'].count()    
    a,b = get_ym(mdcountsall)
    data2 = pd.DataFrame()
    data2['date'] = get_ym_string(a,b)
    data2['outcome'] =  mdcountsall.reset_index()['outcome'].astype(int) 

    #for pivot table
    #data2['hours'] =  d.astype(int) 
    #data2['days']  =  c.astype(int) 
    return data2

## heatmap
def createHeatmap(piv,title="") :
    plt.figure(figsize=(24,8))
    plt.title(title)
    ax = sns.heatmap(piv, square=True)
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=0 )
    plt.tight_layout()
    plt.show()
    return ax

def createBarplot(md,fx,fy,fontscale,title="") :
    sns.set(style='whitegrid', palette='muted', font_scale=fontscale)
    plt.figure(figsize=(fx,fy))
    plt.title(title)
    ax = sns.barplot(x=md['date'], y=md['outcome'], data=md)
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=0 )
    plt.tight_layout()
    plt.show()
    return ax

# kernel density estimate (KDE) 
def createKDE(data2,fx,fy,fontscale,title="") :
    sns.set(style='whitegrid', palette='muted', font_scale=fontscale)
    plt.figure(figsize=(fx,fy))
    plt.title(title)
    # Histogram 
    #ax = sns.distplot(a=data2['outcome'], kde=False)
    ax = sns.kdeplot(data=data2['outcome'], shade=True)
    plt.tight_layout()
    plt.show()
    return ax

In [4]:
def createLineplot(md,fx,fy,fontscale,title="",skip=0,rot=90) :
    sns.set(style='whitegrid', palette='muted', font_scale=fontscale)
    plt.figure(figsize=(fx,fy))
    plt.title(title)
    ax = sns.lineplot(x=md['date'], y=md['value'], data=md)
    for ind, label in enumerate(ax.get_xticklabels()):
        if ind % skip == 0:  # every 10th label is kept
            label.set_visible(True)
            label.set_rotation(rot)
        else:
            label.set_visible(False)
    #plt.setp( ax.xaxis.get_majorticklabels(), rotation=90 )
    plt.tight_layout()
    plt.show()
    return ax

#ax = createLineplot(pfall,16,10,1.4,title="")
#label(ax,500,80)

Time Series Anomaly Detection with LSTM Autoencoders

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [6]:
# functions

OUTCOME = 'close'

TIME_STEPS = 24
#TIME_STEPS = 30
#TIME_STEPS = 720
#TIME_STEPS = 168
#TIME_STEPS = 336

# setup data (current)
def createDataframe(pfall) :
    data3 = createData(pfall,0,2)
    df = pd.DataFrame()
    df[OUTCOME] = data3['outcome']
    df.set_index(data3['date'], inplace=True)
    return df

def getTrainAndTest(df,TRAIN_SIZE) :
    train_size = int(len(df) * TRAIN_SIZE)
    test_size = len(df) - train_size
    train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]
    print("train.shape: ",train.shape, "test.shape: ", test.shape)
    return train, test

def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

def initmodel():
    model = keras.Sequential()
    model.add(keras.layers.LSTM(
        units=64, 
        input_shape=(X_train.shape[1], X_train.shape[2])
    ))
    model.add(keras.layers.Dropout(rate=0.2))
    model.add(keras.layers.RepeatVector(n=X_train.shape[1]))
    model.add(keras.layers.LSTM(units=64, return_sequences=True))
    model.add(keras.layers.Dropout(rate=0.2))
    model.add(keras.layers.TimeDistributed(keras.layers.Dense(units=X_train.shape[2])))
    model.compile(loss='mae', optimizer='adam')
    return model

def testScoreDF(model, THRESHOLD) : 
    X_test_pred = model.predict(X_test)
    test_mae_loss = np.mean(np.abs(X_test_pred - X_test), axis=1)

    test_score_df = pd.DataFrame(index=test[TIME_STEPS:].index)
    test_score_df['loss'] = test_mae_loss
    test_score_df['threshold'] = THRESHOLD
    test_score_df['anomaly'] = test_score_df.loss > test_score_df.threshold
    test_score_df[OUTCOME] = test[TIME_STEPS:][OUTCOME]
    return test_score_df

In [7]:
missingSchemaids = [ 1480883705,  -404024316,  -183769575,  2031641327, -1576843338,
                     -660710506,  -208259125,  -650561809,   -39609584, -1115728007]

def withoutMissingSchemas(pf):
    return pf[~pf['schemaid'].isin(missingSchemaids)]

def getHashValues(pf):
    return pd.unique(pf['hashvalue'])

# max timestamp for splits
def getMaxTimestamp(pf):
    return pf.loc[pf['timestamp'].idxmax()]['timestamp']

def getCountDF(pf,column,hashes):
    dft = pd.DataFrame(columns=[column, 'count'])
    i=0
    for hash in hashes:
        pfall=pf[pf[column] == hash]
        num=len(pd.unique(pfall['value']))
        dft.loc[i] = [hash] + [num]
        i=i+1
    return dft.sort_values('count')

# Start

In [8]:
import numpy as np
import pandas as pd

import findspark
findspark.init()

import pyspark
import random

from pyspark import SparkContext
from pyspark.sql import SQLContext

#sc = pyspark.SparkContext(appName="Pi")

#sc = SparkContext()
#sqlContext = SQLContext(sc)

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

sparkSession = SparkSession.builder.config('spark.local.dir', '/tmp').config("spark.executor.memory", "8g").config("spark.driver.memory", "8g").config("spark.driver.maxResultSize", "0").appName("example-pyspark-read-and-write").getOrCreate()


In [10]:
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/stage1/flat_7/*')

In [ ]:
#df.show()
pf = df.limit(1000000).dropDuplicates().orderBy('timestamp').toPandas()
getMaxTimestamp(pf)

In [ ]:
# 1961237651 , 632085806
#pf[(pf['hashvalue'] == 1961237651) & (pf['timestamp'] == 1583856953)] 
pf.head()

In [ ]:
tb = pf[(pf['hashvalue'] == 632085806) & (pf['timestamp'] == 1583856953)] 
tb

In [ ]:
def getMissingSchemaids(pf):
    a = np.array([])
    for hash in hashes:  
        if len(pf[pf['hashvalue'] == hash]) != len(pd.unique(pf[pf['hashvalue'] == hash]['timestamp'])):
            value = pd.unique(pf[pf['hashvalue'] == hash]['schemaid'])
            if (value in a) == False:
                a = np.append(a,value)
    return a.astype(int)

def usedcolumns(tb):
    col = []
    for column in tb.columns:
        if tb.iloc[0][column] != None :
            col.append(column)
    return col

def keepcolumns(tb,keep):
    for column in tb.columns:
        if column not in keep :
            del(tb[column])

            
            
#used = usedcolumns(tb)
#testdel(tb,used)
#tb

In [ ]:
#pf.head()
#pf[pf['hashvalue'] == 1636706336].head()
#,pf[pf['hashvalue'] == -1407530683].iloc[1]
len(pd.unique(pf['hashvalue']))
hashes = pd.unique(pf['hashvalue'])

#len(pf[pf['hashvalue'] == hashes[5]]),len(pd.unique(pf[pf['hashvalue'] == hashes[5]]['timestamp']))


In [ ]:
pf['schemaid'] = pf['schemaid'].astype(int)

In [ ]:
import numpy as np


  
#missingSchemaids = getMissingSchemaids(pf)  
#missingSchemaids

In [ ]:
pf = withoutMissingSchemas(pf)
hashes = getHashValues(pf)
counts = getCountDF(pf,hashes)

In [ ]:
getMaxTimestamp(pf)

In [ ]:
#
cat_features
dataall['service'] = dataall['service'].astype(str)
dataall['__name__'] = dataall['__name__'].astype(str)
dataall['instance'] = dataall['instance'].astype(str)

In [ ]:
enc = encodeall(dataall,cat_features)

In [ ]:
enc

In [ ]:
pfall=pf[pf['hashvalue'] == dft.loc[13844]['hashvalue']].reset_index()
pfall['value'] = pfall['value'].astype(int)

In [ ]:
pfall.head()

In [ ]:
hash=360972192
len(pf[pf['hashvalue'] == hash]),len(pd.unique(pf[pf['hashvalue'] == hash]['timestamp']))
pf[pf['hashvalue'] == hash]

In [ ]:
pf.columns

# Create encoders

In [11]:
#https://stackoverflow.com/questions/50041551/tell-labelenocder-to-ignore-new-labels

import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.validation import column_or_1d

class TolerantLabelEncoder(LabelEncoder):
    def __init__(self, ignore_unknown=False,
                       unknown_original_value='unknown', 
                       unknown_encoded_value=-1):
        self.ignore_unknown = ignore_unknown
        self.unknown_original_value = unknown_original_value
        self.unknown_encoded_value = unknown_encoded_value

    def transform(self, y):
        check_is_fitted(self, 'classes_')
        y = column_or_1d(y, warn=True)

        indices = np.isin(y, self.classes_)
        if not self.ignore_unknown and not np.all(indices):
            raise ValueError("y contains new labels: %s" 
                                         % str(np.setdiff1d(y, self.classes_)))

        y_transformed = np.searchsorted(self.classes_, y)
        y_transformed[~indices]=self.unknown_encoded_value
        return y_transformed

    def inverse_transform(self, y):
        check_is_fitted(self, 'classes_')

        labels = np.arange(len(self.classes_))
        indices = np.isin(y, labels)
        if not self.ignore_unknown and not np.all(indices):
            raise ValueError("y contains new labels: %s" 
                                         % str(np.setdiff1d(y, self.classes_)))

        y_transformed = np.asarray(self.classes_[y], dtype=object)
        y_transformed[~indices]=self.unknown_original_value
        return y_transformed

In [12]:
def createEncoders(dataall,columns):
    for column in columns:
        le = TolerantLabelEncoder(ignore_unknown=True)
        #le.fit([1, 2, 2, 6])
        le.fit(dataall[column])
        LabelEncoder()
        print(le.classes_)
        np.save(column + '.npy', le.classes_)
        
def encode(dataall,columns):
    # save np.load
    np_load_old = np.load

    # modify the default parameters of np.load
    np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

    for column in columns:
        encoder = TolerantLabelEncoder(ignore_unknown=True)
        encoder.classes_ = np.load(column + '.npy')
        dataall[column] = encoder.transform(dataall[column]) 

    # restore np.load for future normal usage
    np.load = np_load_old
    
def getEncoder(column):
    # save np.load
    np_load_old = np.load

    # modify the default parameters of np.load
    np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

    encoder = TolerantLabelEncoder(ignore_unknown=True)
    encoder.classes_ = np.load(column + '.npy')
        
    # restore np.load for future normal usage
    np.load = np_load_old
    return encoder

In [13]:
# split instance name

def igroup(x):
    if x[0:3] == 'cls':
        res = x.split('0')
        return res[0]
    return x
    

def inode(x):
    if x[0:3] == 'cls':
        res = x.split('0')
        return res[1].split(':')[0]
    return '1'
    

# TMP

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import numpy as np

le = LabelEncoder()
#le.fit([1, 2, 2, 6])
le.fit(dataall['adapter'])
LabelEncoder()
le.classes_
#np.save('classes.npy', le.classes_)

In [ ]:
#le.transform(dataall['adapter']) 
#dataall['adapter']

In [ ]:
encoder = LabelEncoder()
encoder.classes_ = np.load('classes.npy')
encoder.transform([1, 1, 2, 6,6,1]) 

In [14]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

cat_features = ['service', '__name__','instance']

def encodeall(pfall,cat_features):
    #Prepping categorical variables
    from sklearn.preprocessing import LabelEncoder
    encoder = LabelEncoder()
    # Apply the label encoder to each column
    encodedpfall = pfall[cat_features].apply(encoder.fit_transform)
    return encodedpfall
  
def getpfbasic(pfall) :   
    #pfall = df2.limit(5000000).toPandas()    
    #print(pfall.loc[pfall['CSTARTTIME'].idxmax()]['CSTARTTIME'],len(pfall.index))
    pfall = pfall.assign(outcome=(~( ((pfall['CSTATUS'] == 'PENDING') & (pfall['CSERVICE'] == 'InvoicePortal')) | ((pfall['CSTATUS'] == 'PENDING') & (pfall['CSERVICE'] == 'IDS')) | (pfall['CSTATUS'] == 'SUCCESS') | (pfall['CSTATUS'] == 'SUCCESS_DOWNLOADED') | (pfall['CSTATUS'] == 'SUCCESS_POLLQUEUE'))).astype(int))
    converttimestampcolumnn(pfall,'CSTARTTIME')
    pfall['CGLOBALMESSAGEID'] = pfall['CGLOBALMESSAGEID'].apply(hash)
    #pfall['CSENDERENDPOINTID'] = pfall['CSENDERENDPOINTID'].astype(str)   
    #pfall['CRECEIVERENDPOINTID'] = pfall['CRECEIVERENDPOINTID'].astype(str)
    pfall['CMESSAGETAT2'] = pfall['CMESSAGETAT2'].astype(int)
    pfall['CSLATAT'] = pfall['CSLATAT'].astype(int)
    pfall['CINBOUNDSIZE'] = pfall['CINBOUNDSIZE'].astype(int)
    return pfall

def getpf(df2) :    
    pfall = getpfbasic(df2)
    encodedpfall = encodeall(pfall,cat_features)
    dataall = pfall[['CGLOBALMESSAGEID','CMESSAGETAT2','CSLATAT','CINBOUNDSIZE', 'outcome']].join(encodedpfall)
    adddatecolumns(dataall,pfall)    
    return dataall

def astype(pfall,selected,newtype):
    for each in selected:
        pfall[each] = pfall[each].astype(newtype)


In [15]:
import numpy as np
import hashlib
from pyspark.sql.functions import when, lit, col

selected = ['date', 'timestamp', 'value', 'product', 'service', '__name__','instance',  'schemaid', 'hashvalue']
#selected = ['date','timestamp','value','product','service','__name__','instance','job','le','port','schemaid','hashvalue']
#selected = ['timestamp', 'value', 'schemaid', 'hashvalue']

def getpfall(df,selected) :
    #pfall = df.limit(5000000).toPandas()  
    pfall = df.toPandas() 
    pfall['timestamp'] = pfall['timestamp'].astype(int)
    pfall['schemaid'] = pfall['schemaid'].astype(int)
    pfall['hashvalue'] = pfall['hashvalue'].astype(int)
    #pfall['value'] = pfall['value'].astype(float)
    #for each in selected:
    #    pfall[each] = pfall[each].astype(str)
    if len(pfall) == 0:
        return pfall,0,0
    #used = usedcolumns(pfall)
    #keepcolumns(pfall,used)
    return pfall, int(pfall.loc[pfall['timestamp'].astype(int).idxmax()]['timestamp']),len(pfall.index)        

def gettest(to,selected) :
    df2 = df.withColumn('timestamp', col('timestamp').cast('long')).filter(col("timestamp") < to ).select(selected).dropDuplicates().orderBy('timestamp')    
    return getpfall(df2,selected)

def getdata_lt(to,selected=[], filter_names = "") :
    if len(filter_names) > 0:
        print (len(filter_names))
        df2 = df.filter(col("__name__") == filter_names ).withColumn('timestamp', col('timestamp').cast('long')).filter(col("timestamp") < to ).dropDuplicates().orderBy('timestamp') 
    else:
        df2 = df.withColumn('timestamp', col('timestamp').cast('long')).filter(col("timestamp") < to ).dropDuplicates().orderBy('timestamp')  
    #df2 = df.filter(col("__name__") == 'bis_adapter_finished_processes_total' ).withColumn('timestamp', col('timestamp').cast('long')).filter(col("timestamp") < to ).select(selected).dropDuplicates().orderBy('timestamp') 
    #dataall = getpf(df2)    
    #return dataall
    return getpfall(df2,selected)
    #return df2
    

# >=from  <=to    
def getdata_ft(_from,_diff,selected=[], filter_names = "") :
    to = _from + _diff
    if len(filter_names) == 0:
        print(_from,to)
        df2 = df.withColumn('timestamp', col('timestamp').cast('long')) \
                .filter(col("timestamp") >= _from ) \
                .filter(col("timestamp") <= to ) \
                .filter(~df['schemaid'].isin(*missingSchemaids)) 
                #.orderBy('timestamp') 
        print((df2.count(), len(df2.columns)))
    else:    
        df2 = df.filter(col("__name__") == filter_names ).withColumn('timestamp', col('timestamp').cast('long')).filter(col("timestamp") >= _from ).filter(col("timestamp") <= to ).dropDuplicates().orderBy('timestamp') 
    return getpfall(df2,selected)

# >from  <=to   
def getdata_gt(_from,_diff,selected=[], filter_names = "") :
    to = _from + _diff
    if len(filter_names) == 0:
        print(_from,to)
        df2 = df.withColumn('timestamp', col('timestamp').cast('long')) \
                .filter(col("timestamp") > _from ) \
                .filter(col("timestamp") <= to ) \
                .filter(~df['schemaid'].isin(*missingSchemaids)) 
                #.orderBy('timestamp') 
        print((df2.count(), len(df2.columns)))
    else:    
        df2 = df.filter(col("__name__") == filter_names ).withColumn('timestamp', col('timestamp').cast('long')).filter(col("timestamp") >= _from ).filter(col("timestamp") <= to ).dropDuplicates().orderBy('timestamp') 
    return getpfall(df2,selected)

In [ ]:
#selected = ['__name__']
#df4 = df.select(selected).dropDuplicates().toPandas() 
#len(df4)

In [ ]:
#missingSchemaids
df.show(1)

In [ ]:
row1 = df.agg({"timestamp": "min"}).collect()[0]
row2 = df.agg({"timestamp": "max"}).collect()[0]


In [ ]:
print (row1)
dfminTimestamp = row1["min(timestamp)"]
print (row2)
dfmaxTimestamp = row2["max(timestamp)"]

dfrowCount    = df.count()
dfcolumnCount = len(df.columns)

In [ ]:
_from =  dfminTimestamp
to = _from + 100
timestamp_diff = 3000

df2 = getdata_ft(_from,timestamp_diff,selected=[], filter_names = "")

#df2.show()
#print((df2.count(), len(df2.columns)))

In [20]:
withoutColumns = ['date', 'timestamp', 'value','instance','schemaid', 'hashvalue']
columns = df.limit(1).toPandas().columns
columns = columns[~columns.isin(withoutColumns)]

In [16]:
from datetime import time
import datetime as dt
import calendar
import pytz
de = pytz.timezone('Europe/Berlin')

def date(x):
    return  dt.datetime.fromtimestamp(float(x), tz=de)


def adddatecolumns(data,pf,column) :
    data['year'] = pf[column].apply(lambda x: date(x).date().year)
    data['month'] = pf[column].apply(lambda x: date(x).date().month)
    data['day'] = pf[column].apply(lambda x: date(x).date().day)
    data['hour'] = pf[column].apply(lambda x: date(x).time().hour)
    data['minute'] = pf[column].apply(lambda x: date(x).time().minute)
    #data['second'] = pf[column].apply(lambda x: x.time().second)
    #data['microsecond'] = pf[column].apply(lambda x: x.time().microsecond)

def converttimestampcolumnn(pf,tsc) :
    pf[tsc] = pf[tsc].apply(lambda x: dt.datetime.fromtimestamp(float(x) / 1e3))

In [17]:
def encodeDataframe(dataall):
    astype(dataall,columns,str)
    encode(dataall,columns)

    del dataall['date']
    #del dataall['timestamp']
    
    astype(dataall,['instance'],str)
    dataall['igroup'] = dataall['instance'].apply(lambda x: igroup(x))
    dataall['inode']  = dataall['instance'].apply(lambda x: inode(x))

    #createEncoders(dataall,['igroup'])
    encode(dataall,['igroup'])
    astype(dataall,['inode'],int)
    del dataall['instance']

    # convert timestamp to datetime and add column date
    import calendar
    import pytz
    de = pytz.timezone('Europe/Berlin')
    #dataall['date'] = dataall['timestamp'].apply(lambda x: dt.datetime.fromtimestamp(float(x), tz=de))
    adddatecolumns(dataall,dataall,'timestamp')
    #del dataall['date'] 
    return dataall

In [ ]:
dftest = df.select('timestamp').toPandas() 

In [ ]:
dftest['date'] = dftest['timestamp'].apply(lambda x: dt.datetime.fromtimestamp(float(x), tz=de))

In [ ]:
adddatecolumns(dftest,dftest,'date')

In [ ]:
dftest

In [ ]:
df2[0].head()
import calendar
import pytz
de = pytz.timezone('Europe/Berlin')
dt.datetime.fromtimestamp(float(df2[0].iloc[0]['timestamp']), tz=de)

In [ ]:
dataall = encodeDataframe(df2[0])
timestamp = df2[1]
timestamp_diff = 1000

In [ ]:
#dataall = withoutMissingSchemas(dataall)
#dataall.head()

dataall.to_parquet('/tmp/myfile_7_' + str(timestamp) + '.parquet', engine='fastparquet', compression='GZIP')

In [ ]:
#!rm /tmp/*.parquet
#!ls -lht /tmp/*.parquet

In [ ]:
# max timestamp
dataall.loc[dataall['timestamp'].idxmax()]['timestamp'], timestamp

In [18]:
def walk(timestamp,max_cycles=1,timestamp_diff=1000):
    cycle = 0
    while True:
        #next
        df2 = getdata_gt(timestamp,timestamp_diff)
        encodeDataframe(df2[0])

        #merge and prepare for next step
        #dataall = dataall.append(df2[0], ignore_index=True)
        timestamp = df2[1]
        print(timestamp,df2[2])
        df2[0].to_parquet('/tmp/myfile_7_' + str(timestamp) + '.parquet', engine='fastparquet', compression='GZIP')
        
        if df2[2] == 0:
            break  
        cycle = cycle + 1
        if cycle == max_cycles:
            break
    return timestamp

In [21]:
timestamp = 1584187648
timestamp = walk(timestamp,1000,3000)

1584187648 1584190648
(2715976, 58)
1584190631 2715976
1584190631 1584193631
(2741692, 58)
1584193621 2741692
1584193621 1584196621
(2741636, 58)
1584196610 2741636
1584196610 1584199610
(2715982, 58)
1584199591 2715982
1584199591 1584202591
(2741722, 58)
1584202581 2741722
1584202581 1584205581
(2741692, 58)
1584205569 2741692
1584205569 1584208569
(2741654, 58)
1584208559 2741654
1584208559 1584211559
(2715974, 58)
1584211539 2715974
1584211539 1584214539
(2741718, 58)
1584214531 2741718
1584214531 1584217531
(2741618, 58)
1584217523 2741618
1584217523 1584220523
(2715930, 58)
1584220505 2715930
1584220505 1584223505
(2741876, 58)
1584223494 2741876
1584223494 1584226494
(2741606, 58)
1584226485 2741606
1584226485 1584229485
(2715461, 58)
1584229466 2715461
1584229466 1584232466
(2741902, 58)
1584232459 2741902
1584232459 1584235459
(2742026, 58)
1584235450 2742026
1584235450 1584238450
(2741890, 58)
1584238441 2741890
1584238441 1584241441
(2716340, 58)
1584241424 2716340
1584241424

KeyboardInterrupt: 

In [ ]:
dataall

In [ ]:
len(pd.unique(dataall['hashvalue']))

In [ ]:
df3 = getdata_ft(timestamp,timestamp_diff,selected=[], filter_names = "")

In [ ]:
pd.unique(df3[0]['timestamp'])

In [ ]:
start_lt = 1583856983
#start_lt = 1585498272

timestamp_diff = 5000
dataall2 = getdata_lt(start_lt,selected)
dataall = dataall2[0]
timestamp = dataall2[1]

In [ ]:
#timestamp_diff = 10
def walk(dataall,timestamp,max_cycles=1):
    cycle = 0
    while True:
        dataall2 = getdata_gt(timestamp ,timestamp_diff,selected)
        print(dataall2[2])
        if dataall2[2] == 0:
            break  
        dataall = dataall.append(dataall2[0], ignore_index=True)
        cycle = cycle + 1
        if cycle == max_cycles:
            break
        timestamp = dataall2[1]
    return dataall,dataall2[1],dataall2[2]

#timestamp = dataall2[1]
dataall,timestamp,length = walk(dataall,timestamp,1000)

In [ ]:
dataall.head()

In [ ]:
timestamp_diff = 10
def walk(dataall2,dataall,max_cycles=1):
    cycle = 0
    while True:
        dataall2 = getdata_gt(dataall2[1] ,timestamp_diff,selected)
        print(dataall2[2])
        if dataall2[2] == 0:
            break  
        dataall = dataall.append(dataall2[0], ignore_index=True)
        cycle = cycle + 1
        if cycle == max_cycles:
            break
    return dataall,dataall2[1],dataall2[2]

dataall,timestamp,length = walk(dataall2,dataall,1)
dataall = dataall2[0]

In [ ]:
# testEncoding
#pd.unique(dataall['__name__'])
#astype(dataall,['adapter'],str)
#pd.unique(dataall['__name__'])
dataall.dtypes

In [ ]:
# convert dataall to dataframe and store it to hdfs
#dataall
#dfm = sparkSession.createDataFrame(dataall)
#dfm.write.parquet('hdfs://172.30.17.145:8020/user/admin/bis_adapter_finished_processes_total_7.parquet')

import os
#os.environ['http_proxy'] = "http://172.30.12.56:3128" 
#os.environ['https_proxy'] = "https://172.30.12.56:3128"    
#!conda install -y -c conda-forge fastparquet
dataall.to_parquet('/tmp/myfile_7.parquet', engine='fastparquet', compression='GZIP')

In [ ]:

!ls -lh /tmp/myfile*.parquet

#pd.read_parquet('/tmp/myfile.parquet', engine='fastparquet')

In [ ]:
#pft = dataall2.toPandas() 
#dataall
#getMaxTimestamp(dataall)
#dataall = withoutMissingSchemas(dataall)

used = usedcolumns(dataall)
#keepcolumns(dataall,used)
used

In [ ]:
def getVariableUniqueColums(dataall):
    col = []
    for column in dataall.columns:
        size = len(pd.unique(dataall[column]))
        #print(column,size)
        if size > 1:
            col.append(column)
    return col

keep = getVariableUniqueColums(dataall)
keepcolumns(dataall,keep)
keep

In [ ]:
dataall.head()
#pd.unique(dataall['adapter'])

In [ ]:
#used_columns = ['__name__','adapter','job','logical_system','instance','product','service']
used_columns = ['__name__','job','instance','product','service']
astype(dataall,used_columns,str)

In [ ]:
withoutColumns = ['date', 'timestamp', 'value','instance','schemaid', 'hashvalue']
columns = df.limit(1).toPandas().columns
columns = columns[~columns.isin(withoutColumns)]

In [ ]:
for column in columns:
    print(column)
    df4 = df.select(column).dropDuplicates().toPandas() 
    df4[column] = df4[column].astype(str)
    createEncoders(df4,[column])

In [ ]:
column = 'instance'
df4 = df.select(column).dropDuplicates().toPandas() 
df4[column] = df4[column].astype(str)
#createEncoders(df4,[column])

In [ ]:
#astype(dataall,['instance'],str)
df4['igroup'] = df4['instance'].apply(lambda x: igroup(x))
df4['inode']  = df4['instance'].apply(lambda x: inode(x))

In [ ]:
createEncoders(df4,['igroup'])

In [ ]:
encoder = getEncoder(columns[8])
print(encoder.inverse_transform([5]))


In [ ]:
selected = ['__name__']
df4 = df.select(selected).dropDuplicates().toPandas() 
len(df4)

In [ ]:
createEncoders(dataall,used_columns)

In [ ]:
encode(dataall,used_columns)

In [ ]:
f = 'bis_adapter_finished_processes_total'
#encoder = getEncoder(dataall,'__name__')
#encoder = getEncoder('__name__')
encoder = getEncoder(used_columns[0])
print(encoder.inverse_transform([413]))
#print(encoder.transform([f]))
#encoder.classes_
#createEncoders(dataall,['service'])

In [ ]:
en = TolerantLabelEncoder(ignore_unknown=True)
en.fit(['a','b'])

print(en.transform(['a', 'c', 'b']))
# Output: [ 0 -1  1]

print(en.inverse_transform([-1, 0, 1]))
# Output: ['unknown' 'a' 'b']

en.classes_

In [ ]:
#np.save('test' + '.npy', en.classes_)
getEncoder('test').classes_

In [ ]:
# create and save encoders / cannot adapt to new labels
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import numpy as np

def createEncoders(dataall,columns):
    for column in columns:
        le = LabelEncoder()
        #le.fit([1, 2, 2, 6])
        le.fit(dataall[column])
        LabelEncoder()
        le.classes_
        np.save(column + '.npy', le.classes_)
    
def encode(dataall,columns):
    # save np.load
    np_load_old = np.load

    # modify the default parameters of np.load
    np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

    for column in columns:
        encoder = LabelEncoder()
        encoder.classes_ = np.load(column + '.npy')
        dataall[column] = encoder.transform(dataall[column]) 

    # restore np.load for future normal usage
    np.load = np_load_old
    
createEncoders(dataall,['__name__','adapter','job','logical_system','product','service'])

In [ ]:
import numpy as np

encode(dataall,['__name__','adapter','job','logical_system','product','service'])

In [ ]:
dataall = withoutMissingSchemas(dataall)

In [ ]:
astype(dataall,['instance'],str)
dataall['igroup'] = dataall['instance'].apply(lambda x: igroup(x))
dataall['inode']  = dataall['instance'].apply(lambda x: inode(x))

In [ ]:
createEncoders(dataall,['igroup'])
encode(dataall,['igroup'])

In [ ]:
# convert timestamp to datetime and add column date
import calendar
import pytz
de = pytz.timezone('Europe/Berlin')
dataall['date'] = dataall['timestamp'].apply(lambda x: dt.datetime.fromtimestamp(float(x), tz=de))

In [ ]:
adddatecolumns(dataall,dataall,'date')

In [ ]:
dataall.head()

In [ ]:
cat_features=['group']
enc = encodeall(dataall,cat_features)
dataall['group'] = enc['group']
#dataall['instance'] = enc['instance']
astype(dataall,['group','node','value'],int)
del(dataall['instance']) 
dataall['service'] = encodeall(dataall,['service'])['service']
dataall['logical_system'] = encodeall(dataall,['logical_system'])['logical_system']
dataall['job'] = encodeall(dataall,['job'])['job']
dataall['adapter'] = encodeall(dataall,['adapter'])['adapter']

In [ ]:
#dataall
import calendar
import pytz
res="2020-03-10 16:59:59"
x=1583855999
de = pytz.timezone('Europe/Berlin')
dt.datetime.fromtimestamp(float(x), tz=de)

In [ ]:
#dataall.dtypes
#dataall.head()
dataall[dataall['inode'] == '9']
pd.unique(dataall['igroup'])
#astype(dataall,'date',dt.datetime)

In [ ]:
dataall[(dataall['service'] == 3) & (dataall['job'] == 6) & (dataall['adapter'] == 10) & (dataall['inode'] != 4)]['inode']
#pd.unique(dataall[dataall['job'] == 'MAKTEST']['service'])
#pd.unique(dataall[dataall['job'] == 'MAKPRO']['node'])

In [ ]:
mdcountsall = dataall[(dataall['service'] == 3) & (dataall['job'] == 6) & (dataall['adapter'] == 10) & (dataall['node'] == 3)]
mdcountsall = mdcountsall.groupby(['year','month','day','hour'])['value'].count()    
len(mdcountsall)

In [ ]:
a,b,c,d = get_ymdh(mdcountsall)
data2 = pd.DataFrame()
data2['date'] = get_ymdh_string(a,b,c,d)
data2['value'] =  mdcountsall.reset_index()['value'].astype(int) 

In [ ]:
ax = createLineplot(data2,16,10,1.4,title="",skip=10)
label(ax,10,90)

In [ ]:
 def label(graph,skip,rot) :
    for ind, label in enumerate(graph.get_xticklabels()):
        if ind % skip == 0:  # every 10th label is kept
            label.set_visible(True)
            label.set_rotation(rot)
        else:
            label.set_visible(False)

In [ ]:
#dataall = withoutMissingSchemas(dataall)
hashes = getHashValues(dataall)
countpf = getCountDF(dataall,'hashvalue',hashes).reset_index()
countpf

In [ ]:
b = dataall[dataall['hashvalue'] == countpf.iloc[224]['hashvalue']]

ax = createLineplot(b,16,10,1.4,title="")

In [ ]:
#dataall[dataall['hashvalue'] == -869302958]
row = dataall[dataall['hashvalue'] == countpf.iloc[224]['hashvalue']].iloc[0]

cluster = cluster_instances
cluster_instances = pd.unique(cluster['instance'])

In [ ]:
cluster_instances

In [ ]:
names = pd.unique(dataall['__name__'])
countnames = getCountDF(dataall,'__name__',names).reset_index()
countnames

In [ ]:
#pd.unique(dataall[dataall['__name__'] == 'bis_adapter_finished_processes_total']['schemaid'])
pd.unique(dataall[dataall['schemaid'] == 232746291]['__name__'])

In [ ]:
astype(dataall,cat_features,str)
enc = encodeall(dataall,cat_features)
dataall['service_enc'] = enc['service']
dataall['__name___enc'] = enc['__name__']
dataall['instance_enc'] = enc['instance']

In [ ]:
#dataall[dataall['hashvalue'] == -869302958]
row = dataall[dataall['hashvalue'] == -488244161].iloc[0]
row['service_enc'], row['__name___enc'], row['instance_enc'],row['schemaid']
cluster = dataall[(dataall['service_enc'] == row['service_enc']) &
        (dataall['__name___enc'] == row['__name___enc']) &
        (dataall['schemaid'] == row['schemaid'])]
cluster_instances = pd.unique(cluster['instance_enc'])

In [ ]:
a = cluster[cluster['instance_enc'] == cluster_instances[0]]
pd.unique(a['hashvalue'])
#getMissingSchemaids(cluster)
#1961237651 , 632085806
#cluster[cluster['hashvalue'] == -48824416]

#le   = pd.unique(a['le'])
#port = pd.unique(a['port'])
#a.columns
#pd.unique(a['job'])
#a.head()
#232746291
#missingSchemaids

for column in a.columns:
    print(column,len(pd.unique(a[column])))
    
adapters=pd.unique(a['adapter'])
hashes=pd.unique(a['hashvalue'])

In [ ]:
#.loc[row_indexer,col_indexer] = value 
b=a[(a['hashvalue'] == hashes[4]) ]
#b = a[(a['port'] == port[0]) & (a['le'] == le[2]) ]
b['value'] = b['value'].astype(int)
#pd.unique(b['hashvalue'])
#pd.unique(a['port'] == '8443')
#a['port']
ax = createLineplot(b,16,10,1.4,title="")
label(ax,500,80)
#b
len(b['value'])

# Number messages sent by endpoints

In [ ]:
md = createData_ym(pfall,0,2)
createBarplot(md,24,9,3.0,title="number messages sent by all endpoints")

In [ ]:
md = createData_ym(pfall,0,1)
createBarplot(md,24,9,3.0,title="number messages with errors sent by all endpoints")

# Details for different CSENDERENDPOINTID

In [ ]:
#pd.unique(mdcountsall[1].index.get_level_values(0))
TOP=500000
result = pfall.groupby(['CSENDERENDPOINTID']).count()
data2 = pd.DataFrame()
data2['date'] = result.index.get_level_values(0).astype(str)
data2['outcome'] =  result['outcome'].astype(int)
topsender =  data2[data2['outcome'] > TOP].sort_values('outcome').reset_index()
topsender.columns = ['index', 'CSENDERENDPOINTID', 'outcome']

In [ ]:
#pfall1 = pfall[pfall['CSENDERENDPOINTID'].isin(topsender['date'])]
pfall1 = pfall[pfall['CSENDERENDPOINTID']==int(topsender.iloc[7]['CSENDERENDPOINTID'])]

In [ ]:
title = str(len(topsender)) + " senders with more than " + str(TOP) + " messages (kernel density estimate )"
ax = createKDE(data2,16,8,2,title)
label(ax,1,90)
#topsender.iloc[7]

# Number messages of selected endpoint (Msgs / h)

In [ ]:
print("CSENDERENDPOINTID: " + str(topsender.iloc[7]['CSENDERENDPOINTID']) + ": number messages so far = " + str(topsender.iloc[7]['outcome']))
        

def createHeatmapSeries(pfall, months,category, outcome) :
    for month in months:
        data2 = createData(pfall,month,outcome)
        piv = pd.pivot_table(data2, values="outcome",index=["hours"], columns=["days"], fill_value=0)
        #titlestring = "CSENDERENDPOINTID: " + str(topsender.iloc[7]['CSENDERENDPOINTID']) + ": "+ category + " so far = " + str(topsender.iloc[7]['outcome']) + " , month: " + str(month) 
        titlestring = "CSENDERENDPOINTID: " + str(topsender.iloc[7]['CSENDERENDPOINTID']) + ": "+ category  + " month: " + str(month) 
        sns.set(style='whitegrid', palette='muted', font_scale=1.2)
        createHeatmap(piv, titlestring)
       
createHeatmapSeries(pfall1,[10,11,12,1,2,3,4],'messages', 2)

# Number messages with errors of selected endpoint

In [ ]:
createHeatmapSeries(pfall1, [11,12,1,2,3,4],'messages with errors', 1)

# Anomaly Detection with LSTM Autoencoders (selected SENDERPOINTID)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
df = createDataframe(pfall1)
train, test = getTrainAndTest(df,0.95)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler = scaler.fit(train[[OUTCOME]])
train[OUTCOME] = scaler.transform(train[[OUTCOME]])
test[OUTCOME] = scaler.transform(test[[OUTCOME]])

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train[[OUTCOME]], train.close, TIME_STEPS)
X_test, y_test = create_dataset(test[[OUTCOME]], test.close, TIME_STEPS)
print(X_train.shape)

model = initmodel()

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.1,
    shuffle=False)

X_train_pred = model.predict(X_train)
train_mae_loss = np.mean(np.abs(X_train_pred - X_train), axis=1)

In [ ]:
sns.distplot(train_mae_loss, bins=50, kde=True);

In [ ]:
test_score_df = testScoreDF(model, 1.5)
anomalies     = test_score_df[test_score_df.anomaly == True]

In [ ]:
plt.plot(test_score_df.index, test_score_df.loss, label='loss')
plt.plot(test_score_df.index, test_score_df.threshold, label='threshold')
plt.xticks(rotation=25)
plt.legend();

In [ ]:
#anomalies.head()
#anomalies

In [ ]:
plt.plot(
  test[TIME_STEPS:].index, 
  scaler.inverse_transform(test[TIME_STEPS:].close), 
  label='msg count'
);

ax = sns.scatterplot(
  anomalies.index,
  scaler.inverse_transform(anomalies.close),
  color=sns.color_palette()[3],
  s=152,
  label='anomaly'
)
plt.xticks(rotation=25)
plt.legend();

label(ax,5,80)